In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: AuthenticAMD
cpu family	: 23
model		: 49
model name	: AMD EPYC 7B12
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2249.998
cache size	: 512 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 rdseed adx smap clflushopt clwb sha_ni xsaveopt xsavec xgetbv1 clzero xsaveerptr arat npt nrip_save umip rdpid
bugs		: sysret_ss_attrs null_seg spectre_v1 spectre_v2 spec_store_bypass retbleed smt_rsb srso
bogo

In [1]:
!pip install datasets
!pip install -U "huggingface_hub[cli]"
!pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `kzzzzq` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `kzzzzq`


In [2]:
import re
from tqdm import tqdm
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteriaList, StoppingCriteria
from torch.profiler import profile, record_function, ProfilerActivity
import time
import json

In [5]:
# Using 8-shot Cot as in https://github.com/EleutherAI/lm-evaluation-harness/blob/main/lm_eval/tasks/gsm8k/gsm8k-cot.yamllm-evaluation-harness gsm8k-cot
# Every question in the gsm8k dataset will follow the examplar below:
FEW_SHOT_PROMPT = """ Here are some examples:
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.

Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.

Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8.

Now please answer the following question. Answer the question step by step.
Q: {question}
A:"""
ZERO_SHOT_COT_PROMPT = """
Q: {question} Let's think step by step.
"""


# load the llama 3.2 1B model and quantize to 8 bits
def load_model_and_tokenizer(model_name="meta-llama/Llama-3.2-1B",quantization_scheme=8,device='cpu'):
  print('Loading model and tokenizer...')
  model = AutoModelForCausalLM.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  if quantization_scheme == 8: # can we have lower than 8 bits?
    model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
  model.to(device)
  return model,tokenizer


def extract_answer(output):
  '''
  extract the answer from the decoded output of the model
  first get the text after the final A:, then extract the numeric answer from this
  reference: https://github.com/tianlwang/eval_gsm8k/blob/main/utils.py
  '''
  decoded_output = output.split('A:')[-1].strip()
  regex_pattern = "(-?[$0-9.,]{2,})|(-?[0-9]+)"
  '''
  (-?[$0-9.,]{2,})

    -?: This matches an optional negative sign (the - at the start of a number).

    [$0-9.,]{2,}: This matches strings that are at least 2 characters long and may consist of:

      The dollar sign ($)

      Digits (0-9)

      Commas (,)

      Periods (.)

    Together, this part of the regex matches numeric-like values that may include currency symbols,
    commas, and decimal points, e.g., $12.34, -1,234, or 300.5.

  (-?[0-9]+)

    -?: Again, this matches an optional negative sign.

    [0-9]+: This matches one or more digits.

    Together, this part of the regex matches plain integers (positive or negative), e.g., 42 or -99.
  '''
  regexes_to_ignore =[
        ",",
        "\\$",
        "(?s).*#### ",
        "\\.$"
    ]

  answer = re.findall(regex_pattern, decoded_output) # a list of tuples
  if answer:
    answer = answer[-1]
    if isinstance(answer,tuple):
      answer = [a for a in answer if a]
      answer = answer[0].strip() # get the first element
      for regex in regexes_to_ignore:
        answer = re.sub(regex, "", answer) # remove the patterns in regexes_to_ignore
      return answer
  else:
    return None

# define stopping criteria
class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, keywords_list, tokenizer, start_len):
        #self.keywords_list = keywords_list # a list of strings
        self.tokenizer = tokenizer
        self.start_len = start_len  # Initialize start_len
        self.keywords_list = keywords_list # stop strings

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        current_text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)[self.start_len:] # start from the last answer

        return any(keyword in current_text for keyword in self.keywords_list)

def memory_profiling(model,tokenizer,keywords_list,prompt,device='cpu',latency_measure=1):
  '''
  send the prompt to the model and record the memory footprint

  parameters:
  model: the model
  tokenizer: the tokenizer
  keywords_list: a list of strings, indicating the stopping criteria
  prompt: the formatted prompt
  device: the device to run the model on
  latency_measure: an integer, 0 means generating only one output token, 1 means generating maximum 512 new tokens
  '''

  # define stop criteria list
  stopping_criteria = KeywordsStoppingCriteria(keywords_list,tokenizer,len(prompt)) # define stopping criteria
  stopping_criteria_list = StoppingCriteriaList([stopping_criteria])
  tokenized_prompt = tokenizer(prompt,return_tensors='pt').to(device) # tokenize the prompt

  max_new_tokens = 512 if latency_measure==1 else 1

  with profile(activities=[ProfilerActivity.CPU], profile_memory=True, on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')) as prof:
    model.generate(**tokenized_prompt,max_new_tokens=max_new_tokens,stopping_criteria=stopping_criteria_list)
  #print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

def write_result(model_name,accuracy,total_output_token_count,total_generation_time_l,TTFT_l,latency_measure):
  '''
  please use this function to write the result to a json file after each experiment
  parameters:
  model_name: the name of the model
  accuracy: a float, the accuracy of the model on the dataset
  total_output_token_count: a list, where each item is the total output token of one response
  total_generation_time_l: a list, where each item is the total generation time of one response
  latency_measure: 0 or 1. 0 means to measure the TTFT of the model, 1 means to measure the total generation time of the model
  '''
  s = 'TTFT' if latency_measure==0 else 'Total generation time'
  dic = {'accuracy':accuracy,'total_output_token_count':total_output_token_count,'total_generation_time_l':total_generation_time_l,'TTFT_l':TTFT_l}
  with open(f'{model_name}_{s}_result.json','w') as f:
    f.write(json.dumps(dic))
  # use json.load(open('result.json','r')) to get the result back

a = extract_answer('hello world 115,000.')
print(a)
print(re.sub(',', "", a))

115000
115000


In [ ]:

def evaluate_on_gsm8k(model, tokenizer, template_prompt, dataset_name='gsm8k', limit=1.0, latency_measure=1, device='cpu', verbose=True, do_memory_profiling=False):
  '''
  parameters:
  model: the model
  tokenizer: the tokenizer
  template_prompt: the prompt template, template = FEW_SHOT_PROMPT means 8-shot Cot, template = ZERO_SHOT_COT_PROMPT means zero-shot Cot
  dataset_name: the name of the dataset
  limit: a float number between 0 and 1, indicating the percentage of the dataset to be used, or an integer that indicates which samples to use
  latency_measure: an integer, 0 means to measure the TTFT of the model, 1 means to measure the total generation time of the model
           for an explanation of TTFT and total generation time, see
           https://symbl.ai/developers/blog/a-guide-to-llm-inference-performance-monitoring/#:~:text=There%20are%20several%20ways%20to%20measure%20a%20model%E2%80%99s,Per%20Output%20Token%20%28TPOT%29%203%20Total%20generation%20time
  device: the device to run the model on
  verbose: a boolean, whether to print the accuracy, total output token count, total generation time, and TTFT
  do_memory_profiling: a boolean, whether to do memory profiling of the model
             the memory footprint is obtained by sending one sample to the model and recording the memory footprint
             during the inference.

  returns:
  accuracy: a float, the accuracy of the model on the dataset. Discarded if latency_measure=0
  total_output_token: a list, where each item is the total output token of one response. Discarded if latency_measure=0
  total_generation_time_l: a list, where each item is the total generation time of one response. Discarded if latency_measure=0
  TTFT_l: a list, where each item is the TTFT of one response. Discarded if latency_measure=1

  '''


  # evalute the model on gsm8k dataset

  assert latency_measure in [0,1], 'latency_measure must be 0 or 1'
  assert isinstance(limit,(int,float)), 'not a valid limit'
  assert isinstance(verbose,bool), 'not a valid verbose'
  assert isinstance(do_memory_profiling,bool), 'not a valid memory_profiling'

  # define stop criteria list
  keywords_list = ['Q:','\n']


  # load dataset
  print(f'loading {dataset_name} dataset')
  dataset = load_dataset(dataset_name, "main", split='test')
  dataset_len = len(dataset)
  if isinstance(limit,int):
    assert 0 < limit <= dataset_len, 'limit must be less than or equal to the length of the dataset and greater than 0'
    dataset_len = limit
  else:
    assert 0 < limit <= 1, 'limit must be greater than 0 and less than or equal to 1'
    dataset_len = int(limit * dataset_len)

  # peek the first item of the dataset
  if verbose:
    print('peek the first item of the dataset')
    print(dataset[0])
    print('\n')


  correct_count = 0
  total_output_token_count = 0
  total_generation_time_l = []
  TTFT_l = []
  #peak_memory_usage_l = []

  #tracemalloc.start()
  for i in tqdm(range(dataset_len)):
    sample = dataset[i]
    #print(sample)

    prompt = template_prompt.format(question=sample['question'])# concatenate the template prompt to the question

    if do_memory_profiling and i==0:
      memory_profiling(model,tokenizer,keywords_list,prompt,device,latency_measure=latency_measure) # only do memory profiling on the first sample


    stopping_criteria = KeywordsStoppingCriteria(keywords_list,tokenizer,len(prompt)) # define stopping criteria
    stopping_criteria_list = StoppingCriteriaList([stopping_criteria])

    answer = sample['answer'].split('####')[-1].strip() # ground truth is the number after the #### of the answer
    #print(answer)
    tokenized_prompt = tokenizer(prompt,return_tensors='pt').to(device) # tokenize the prompt
    input_token_count = len(tokenized_prompt['input_ids'].flatten()) # token number of this iteration


    max_new_tokens = 512 if latency_measure==1 else 1



    t1 = time.perf_counter()
    output = model.generate(**tokenized_prompt,max_new_tokens=max_new_tokens,stopping_criteria=stopping_criteria_list) # as per the setting given by meta
    t2 = time.perf_counter()



    processing_time = t2-t1 # in s, total generation time of one response if latency_measure==1, TTFT if latency_measure=0

    total_output_token_count += len(output.flatten())-input_token_count # total output token count of one response

    total_generation_time_l.append(processing_time)
    TTFT_l.append(processing_time)


    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True) # decode the output
    if verbose and i<5:
      print('the first decoded output is:')
      print(decoded_output) # just for checking

      #pass


    decoded_output = extract_answer(decoded_output) # extract the numerical answer from the decoded output
    if verbose and i<5:
      print('extracted answer:',decoded_output)
      print('answer:',answer)
    if decoded_output == answer:
      correct_count += 1
    #break


  accuracy = correct_count/dataset_len # accuracy of the model on the dataset
  if latency_measure==1:
    print('\n')
    print(f'Accuracy: {accuracy}')
    print(f'Total output token count: {total_output_token_count}')
    print(f'Total processing time: {sum(total_generation_time_l)} s')

  else:
    print('\nplease disregard the accuracy, total_output_token_count, total_generation_time_l in the output')
  if do_memory_profiling:
      print('check the log file to see memory footprint')

  return accuracy,total_output_token_count,total_generation_time_l,TTFT_l





In [ ]:
device = 'cpu'
# get the model and tokenizer
model,tokenizer = load_model_and_tokenizer(model_name='meta-llama/Llama-3.2-1B-Instruct',quantization_scheme=None,device=device)
#print(f"Current memory usage is {current_load_model / (1024*1024)} MB; Peak was {peak_load_model / (1024*1024)} MB")

accuracy,total_output_token_count,total_generation_time_l,TTFT_l = evaluate_on_gsm8k(model,tokenizer,FEW_SHOT_PROMPT,limit=100,latency_measure=1,device=device,verbose=False,do_memory_profiling=True)

Loading model and tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

  0%|          | 0/5 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
100%|██████████| 5/5 [00:42<00:00,  8.49s/it]


please disregard the accuracy, total_output_token_count, total_generation_time_l in the output
check the log file to see memory footprint


In [ ]:
print(f'Accuracy: {accuracy}')
print(f'Total output token count: {total_output_token_count}')
print(f'Total generation time: {total_generation_time_l} s')
print(f'TTFT:{TTFT_l}')


Accuracy: 0.0
Total output token count: 5
Total generation time: [7.085437603000059, 6.650176518999956, 6.967019832000005, 6.636011845999974, 7.436973363999982] s
TTFT:[7.085437603000059, 6.650176518999956, 6.967019832000005, 6.636011845999974, 7.436973363999982]


In [ ]:
# do not run this cell
# cannot visit the local server on Colab. Need to download the log file and see it locally.
# to see the log, create a virtual environment in your machine, run the command:
# pip install torch_tb_profiler
# After installing, run
# tensorboard --logdir=./log
!tensorboard --logdir=./log

2025-03-15 06:00:32.680295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742018432.718915   24218 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742018432.731565   24218 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-15 06:00:37.970967: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

TensorBoard 2.18.0 at http://acfc58c20515:6006/ (Press CTRL+C

In [ ]:
'''
s = 'hello world'
tokenized_prompt = tokenizer(s,return_tensors='pt')
input_token_count = len(tokenized_prompt['input_ids'].flatten())
print(tokenized_prompt)
print(input_token_count)

tracemalloc.start()
output = model.generate(**tokenized_prompt,max_new_tokens=1)
current, peak = tracemalloc.get_traced_memory() # see the memory usage of generating one
tracemalloc.stop()
print(f"Current memory usage is {current / (1024*1024)}MB; Peak was {peak / (1024*1024)}MB")
print(output)
print(len(output.flatten()))
'''

In [ ]:
ZERO_SHOT_COT_PROMPT = """
Q: {question}
Choose among the following options:
{a}
{b}
{c}
{d}
{e}
Let's think step by step:
"""


def evaluate_on_mathqa(model, tokenizer, template_prompt=ZERO_SHOT_COT_PROMPT, dataset_name='allenai/math_qa'):
  # evalute the model on mathqa dataset
  # template = FEW_SHOT_PROMPT means 8-shot Cot
  # template = ZERO_SHOT_COT_PROMPT means zero-shot Cot

  # define stop criteria list
  keywords_list = ['\n']

  # load dataset
  print(f'loading {dataset_name} dataset')
  dataset = load_dataset(dataset_name, "main", split='test',trust_remote_code=True)
  dataset_len = len(dataset)
  dataset_len = 5

  # peek the first item of the dataset
  print('peek the first item of the dataset')
  print(dataset[0])
  print('\n')

  correct_count = 0
  total_token_count = 0
  total_processing_time = 0
  speed_per_itr = []

  for i in tqdm(range(dataset_len)):
    sample = dataset[i]

    options = re.findall(r'[a-zA-Z]\s?\)\s?[^,]+',sample['options'])
    prompt = template_prompt.format(question=sample['Problem'],a=options[0],b=options[1],c=options[2],d=options[3],e=options[4])# concatenate the template prompt to the question

    stopping_criteria = KeywordsStoppingCriteria(keywords_list,tokenizer,len(prompt)) # define stopping criteria
    stopping_criteria_list = StoppingCriteriaList([stopping_criteria])

    #print(prompt)
    answer = sample['correct'] # ground truth
    #print(answer)
    tokenized_prompt = tokenizer(prompt,return_tensors='pt').to('cpu') # tokenize the prompt
    output = model.generate(**tokenized_prompt,max_new_tokens=512,stopping_criteria=stopping_criteria_list) #
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True) # decode the output
    print(decoded_output)
    break
evaluate_on_mathqa(model,tokenizer)

loading allenai/math_qa dataset
peek the first item of the dataset
{'Problem': 'a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 % . what would have been the percentage of profit earned if no discount had been offered ?', 'Rationale': '"giving no discount to customer implies selling the product on printed price . suppose the cost price of the article is 100 . then printed price = 100 ã — ( 100 + 31.1 ) / ( 100 â ˆ ’ 5 ) = 138 hence , required % profit = 138 â € “ 100 = 38 % answer a"', 'options': 'a ) 38 , b ) 27.675 , c ) 30 , d ) data inadequate , e ) none of these', 'correct': 'a', 'annotated_formula': 'subtract(divide(multiply(add(const_100, 31.1), const_100), subtract(const_100, 5)), const_100)', 'linear_formula': 'add(n1,const_100)|subtract(const_100,n0)|multiply(#0,const_100)|divide(#2,#1)|subtract(#3,const_100)|', 'category': 'gain'}




  0%|          | 0/5 [00:51<?, ?it/s]


Q: a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 %. what would have been the percentage of profit earned if no discount had been offered?
Choose among the following options:
a ) 38 
b ) 27.675 
c ) 30 
d ) data inadequate 
e ) none of these
Let's think step by step:
First, find the cost price of the article. Let the cost price be x. Then, the selling price after a 5 % discount is 0.95x. The profit percentage is 31.1 %, which is equal to the profit on the selling price. So, the profit is 0.311x. The cost price can be found by subtracting the profit from the selling price, so x = 0.95x + 0.311x. 0.05x = 0.311x. Divide both sides by 0.05 to get x = 6.2. The selling price after a 5% discount is 0.95(6.2) = 5.86. The profit is 5.86 - 6.2 = -0.34, which is a loss, not a profit. Therefore, option d is incorrect.

